# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770832340691                   -0.53    8.0         
  2   -2.772134985771       -2.89       -1.31    1.0    106ms
  3   -2.772169206523       -4.47       -2.61    1.0    121ms
  4   -2.772170719994       -5.82       -4.11    2.0    122ms
  5   -2.772170722996       -8.52       -4.86    2.0    144ms
  6   -2.772170723014      -10.76       -5.72    1.0    111ms
  7   -2.772170723015      -11.83       -6.45    2.0    140ms
  8   -2.772170723015      -14.15       -7.36    1.0    119ms
  9   -2.772170723015   +  -13.95       -7.74    2.0    135ms
 10   -2.772170723015      -13.76       -8.17    1.0    124ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770755137959                   -0.52    9.0         
  2   -2.772058552167       -2.88       -1.32    1.0    130ms
  3   -2.7

1.7735581570934424

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770597789766                   -0.53    9.0         
  2   -2.772051096590       -2.84       -1.30    1.0    137ms
  3   -2.772082837860       -4.50       -2.65    1.0    110ms
  4   -2.772083415871       -6.24       -3.97    2.0    168ms
  5   -2.772083417746       -8.73       -4.59    2.0    124ms
  6   -2.772083417808      -10.21       -5.51    1.0    128ms
  7   -2.772083417811      -11.52       -6.42    2.0    131ms
  8   -2.772083417811      -13.80       -7.17    2.0    137ms
  9   -2.772083417811   +    -Inf       -8.04    1.0    123ms

Polarizability via ForwardDiff:       1.7725349751148816
Polarizability via finite difference: 1.7735581570934424
